In [1]:
pip install novae

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install 'novae[multimodal,conch]'

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [4]:
import novae
import scanpy as sc
import os
import time
import pandas as pd
import openpyxl

/opt/venv/jupyter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
input_dir = "/workspace/Projects/FM/Final data/H5AD/Visium novae"

# Output directory for embedded files
output_dir = "/workspace/Projects/FM/Final data/Visium_embeddings_novae_finetune"
os.makedirs(output_dir, exist_ok=True)

# Collect all .h5ad files
h5ad_files = [f for f in os.listdir(input_dir) if f.endswith(".h5ad")]

In [6]:
summary_records = []

for f in h5ad_files:
    file_path = os.path.join(input_dir, f)
    print(f"\nProcessing {file_path}")


    adata = sc.read_h5ad(file_path)
    adata.obsm["spatial"] = adata.obsm["spatial"].to_numpy().astype("float32")

    start_time = time.time()
    
    # if you have multiple samples in the same adata object, specify `slide_key`
    novae.spatial_neighbors(adata, radius=200)

    # Option 2: fine-tuning (recommended for better performance)
    model = novae.Novae.from_pretrained("MICS-Lab/novae-human-0")
    model.fine_tune(adata, accelerator="gpu", num_workers=4)
    model.compute_representations(adata, accelerator="gpu", num_workers=4)

    elapsed = time.time() - start_time

    shape = adata.obsm["novae_latent"].shape
    print(f"Embedding shape: {shape}")
    print(f"Time taken: {elapsed:.2f} seconds")

    out_path = os.path.join(output_dir, f.replace(".h5ad", "_novae.h5ad"))
    adata.write(out_path)
    print(f" Saved embeddings → {out_path}")

    summary_records.append({
        "sample": f,
        "n_cells": shape[0],
        "embedding_dim": shape[1],
        "runtime_seconds": round(elapsed, 2),
        "status": "success"
    })

summary_df = pd.DataFrame(summary_records)
excel_path = os.path.join(output_dir, "embedding_runtime_summary.xlsx")
summary_df.to_excel(excel_path, index=False)

print(f"\n Summary saved to {excel_path}")


Processing /workspace/Projects/FM/Final data/H5AD/Visium novae/53430.h5ad


[INFO] (novae.utils.build) Computing graph on 4,271 cells (coord_type=generic, delaunay=True, radius=[0.0, 200.0], n_neighs=None)
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
[INFO] (novae.utils._validate) Preprocessed 1 adata object(s) with sc.pp.normalize_total and sc.pp.log1p (raw counts were saved in adata.layers['counts'])
Computing representations: 100%|██████████| 9/9 [00:02<00:00,  4.40it/s]
Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
Yo

┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name          ┃ Type              ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ cell_embedder │ CellEmbedder      │ 31.3 M │ train │     0 │
│ 1 │ encoder       │ GraphEncoder      │  694 K │ train │     0 │
│ 2 │ augmentation  │ GraphAugmentation │      0 │ train │     0 │
│ 3 │ swav_head     │ SwavHead          │ 32.8 K │ train │     0 │
└───┴───────────────┴───────────────────┴────────┴───────┴───────┘

Trainable params: 989 K                                                                                            
Non-trainable params: 31.1 M                                                                                       
Total params: 32.1 M                                                                                               
Total estimated model params size (MB): 128                                                                        
Modules in train mode: 82                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

/opt/venv/jupyter/lib/python3.10/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

...

┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name          ┃ Type              ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ cell_embedder │ CellEmbedder      │ 31.3 M │ train │     0 │
│ 1 │ encoder       │ GraphEncoder      │  694 K │ train │     0 │
│ 2 │ augmentation  │ GraphAugmentation │      0 │ train │     0 │
│ 3 │ swav_head     │ SwavHead          │ 32.8 K │ train │     0 │
└───┴───────────────┴───────────────────┴────────┴───────┴───────┘

Trainable params: 989 K                                                                                            
Non-trainable params: 31.1 M                                                                                       
Total params: 32.1 M                                                                                               
Total estimated model params size (MB): 128                                                                        
Modules in train mode: 82                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

/opt/venv/jupyter/lib/python3.10/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
Computing representations: 100%|██████████| 7/7 [00:01<00:00,  6.00it/s]


Embedding shape: (3376, 64)
Time taken: 85.32 seconds
 Saved embeddings → /workspace/Projects/FM/Final data/Visium_embeddings_novae_finetune/53432_novae.h5ad

 Summary saved to /workspace/Projects/FM/Final data/Visium_embeddings_novae_finetune/embedding_runtime_summary.xlsx
